##### XLM Roberta fine tuning

In [4]:
!pip install nlpaug
!pip install ktrain
!pip install tensorflow
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 43.7 MB/s eta 0:00:0000:01:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 48.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 30.4 MB/s eta 0:00:00
  Created wheel for ktrain: filename=ktrain-0.41.4-py3-none-any.whl size=25316932 sha256=9e88978f4428574a35e15a16278f036641d3c8a31fd46e58c7ddb50027e13a42
  Stored in directory: /root/.cache/pip/wheels/fa/6a/9c/8a873b38bbd8bc207d33c64726bd18f7ef8

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
import nlpaug.augmenter.word as naw
import pandas as pd

In [4]:
path = '/kaggle/input/multibully/Cyberbully_corrected_emotion_sentiment.xlsx - cyberbully.csv'
df = pd.read_csv(path )
print(df.head())

  Img-Name                                           Img-Text Img-Text-Label  \
0    0.jpg  Shivam @shivamishraa Girls be named naina and ...          Bully   
1    1.jpg  Aaloo ke paranthe is the best breakfast Omelet...       Nonbully   
2    2.jpg     For Boyfriend For Bestfriend DESI ADUKT TROLLS          Bully   
3    3.jpg  You find a new YouTuber He's funny All of his ...       Nonbully   
4    4.jpg  not_shubham14 @mentally_dank Kids at Marine Dr...          Bully   

  Img-Label Text-Label Sentiment   Emotion Sarcasm      Harmful-Score  \
0  Nonbully      Bully  Negative   Disgust     Yes  Partially-Harmful   
1  Nonbully   Nonbully   Neutral     Other      No           Harmless   
2     Bully   Nonbully  Negative  Ridicule      No  Partially-Harmful   
3  Nonbully   Nonbully   Neutral   Sadness      No           Harmless   
4  Nonbully   Nonbully  Negative   Sadness      No  Partially-Harmful   

       Target  ... Unnamed: 14 Unnamed: 15 Unnamed: 16  Unnamed: 17     Bully  \

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [3]:
import pandas as pd
import re
import string
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)  
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) 
    text = re.sub(r'\w*\d\w*', '', text)  
    return text
df['Img-Text'] = df['Img-Text'].astype(str).apply(clean_text)
unwanted_labels = ['|', '11', '22', '32', '21']
df = df[~df['Text-Label'].isin(unwanted_labels)]
label_mapping = {"Bully": 1, "Nonbully": 0}
df['Text-Label'] = df['Text-Label'].map(label_mapping)
df.dropna(subset=['Text-Label'], inplace=True)
df = df.drop_duplicates(subset='Img-Text', keep='first')
df.reset_index(drop=True, inplace=True)
print(df.head())


NameError: name 'df' is not defined

In [6]:
import nltk
import os
import shutil
import zipfile
nltk.download('wordnet', download_dir='/kaggle/working/nltk_data')
nltk.download('omw-1.4', download_dir='/kaggle/working/nltk_data')
nltk.data.path.append('/kaggle/working/nltk_data')
nltk_data_path = "/kaggle/working/nltk_data"
nltk.data.path.append(nltk_data_path)
os.makedirs(nltk_data_path, exist_ok=True)
nltk.download('wordnet', download_dir=nltk_data_path)
nltk.download('omw-1.4', download_dir=nltk_data_path)
shutil.move(os.path.join(nltk_data_path, "corpora/wordnet.zip"), "/kaggle/working/wordnet.zip")
shutil.move(os.path.join(nltk_data_path, "corpora/omw-1.4.zip"), "/kaggle/working/omw-1.4.zip")
with zipfile.ZipFile("/kaggle/working/wordnet.zip", 'r') as zip_ref:
    zip_ref.extractall(nltk_data_path + "/corpora")

with zipfile.ZipFile("/kaggle/working/omw-1.4.zip", 'r') as zip_ref:
    zip_ref.extractall(nltk_data_path + "/corpora")

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
from transformers import XLMRobertaTokenizer, TFXLMRobertaForSequenceClassification
def augment_text(df, augmenter, repetitions=1):
    aug_texts = []
    for _, row in df.iterrows():
        text = row['Img-Text']
        for _ in range(repetitions):
            augmented_text = augmenter.augment(text)
            aug_texts.append((augmented_text, row['Text-Label']))
    return pd.DataFrame(aug_texts, columns=['Img-Text', 'Text-Label'])
augmenter = naw.SynonymAug(aug_src='wordnet')
max_size = df['Text-Label'].value_counts().max()
df_balanced = pd.DataFrame()
for label in df['Text-Label'].unique():
    df_class = df[df['Text-Label'] == label]
    df_class_aug = augment_text(df_class, augmenter, repetitions=max_size // len(df_class) - 1)
    df_balanced = pd.concat([df_balanced, df_class, df_class_aug])
df_balanced.reset_index(drop=True, inplace=True)
df_balanced = df_balanced.sample(frac=1, random_state=123).reset_index(drop=True)
class_names = ['0', '1']

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

def encode_texts(tokenizer, texts, max_length=256):
    input_ids = []
    attention_masks = []

    for text in texts:
        text = str(text)
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf',
        )

        input_ids.append(encoded['input_ids'][0])
        attention_masks.append(encoded['attention_mask'][0])

    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_masks)



<ipython-input-7-29d39a0d183a>:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_balanced = pd.concat([df_balanced, df_class, df_class_aug])


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [8]:
X_train, X_temp, y_train, y_temp = train_test_split(df_balanced['Img-Text'], df_balanced['Text-Label'], test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
train_inputs, train_masks = encode_texts(tokenizer, X_train)
val_inputs, val_masks = encode_texts(tokenizer, X_val)
test_inputs, test_masks = encode_texts(tokenizer, X_test)
train_labels = tf.convert_to_tensor(y_train.to_numpy().astype('int32'))
val_labels = tf.convert_to_tensor(y_val.to_numpy().astype('int32'))
test_labels = tf.convert_to_tensor(y_test.to_numpy().astype('int32'))

In [9]:
model = TFXLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=len(class_names))
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
batch_size = 20
history = model.fit(
    [train_inputs, train_masks],
    train_labels,
    batch_size=batch_size,
    validation_data=([val_inputs, val_masks], val_labels), 
    epochs=20,
    callbacks=[early_stopping]
)
print("Training Loss per Epoch:", history.history['loss'])
print("Training Accuracy per Epoch:", history.history['accuracy'])
print("Validation Loss per Epoch:", history.history['val_loss'])
print("Validation Accuracy per Epoch:", history.history['val_accuracy'])
test_loss, test_accuracy = model.evaluate([test_inputs, test_masks], test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
model.save('/kaggle/working/xlm_roberta_model')

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
287/287 [==============================] - 369s 1s/step - loss: 0.4620 - accuracy: 0.7713 - val_loss: 0.4151 - val_accuracy: 0.7919
Epoch 2/20
287/287 [==============================] - 331s 1s/step - loss: 0.4006 - accuracy: 0.7970 - val_loss: 0.3771 - val_accuracy: 0.8128
Epoch 3/20
287/287 [==============================] - 331s 1s/step - loss: 0.3712 - accuracy: 0.8193 - val_loss: 0.4163 - val_accuracy: 0.8059
Epoch 4/20
287/287 [==============================] - 331s 1s/step - loss: 0.3213 - accuracy: 0.8467 - val_loss: 0.4025 - val_accuracy: 0.8282
Epoch 5/20
287/287 [==============================] - 331s 1s/step - loss: 0.2774 - accuracy: 0.8703 - val_loss: 0.4981 - val_accuracy: 0.8268
Training Loss per Epoch: [0.46199527382850647, 0.40059900283813477, 0.37116095423698425, 0.3212928771972656, 0.2774168848991394]
Training Accuracy per Epoch: [0.7713388204574585, 0.7969977259635925, 0.819340169429779, 0.8467446565628052, 0.8703089356422424]
Validation Loss per Epoch: 

NameError: name 'torch' is not defined

In [ ]:
model.save_pretrained('/kaggle/working/xlm_roberta_model')
!zip -r /kaggle/working/xlm_roberta_model.zip /kaggle/working/xlm_roberta_model


Vision transformer fine tuning

In [1]:
!pip install tensorflow transformers scikit-learn matplotlib seaborn pandas numpy tqdm
!pip install --upgrade torch torchvision transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from transformers import ViTForImageClassification, ViTConfig
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import LabelEncoder
from PIL import Image, UnidentifiedImageError
import os
import glob
import random
from tqdm import tqdm 

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class CustomDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.data = self.data.dropna(subset=["Img-Name"]).reset_index(drop=True)
        self.data["Img-Name"] = self.data["Img-Name"].astype(str)
        self.img_dir = img_dir
        self.transform = transform
        self.label_encoder = LabelEncoder()
        self.data["Img-Label"] = self.label_encoder.fit_transform(self.data["Img-Label"])
        self.valid_indices = self._filter_valid_images()

    def _find_correct_image_path(self, img_name):
        base_name = os.path.splitext(img_name)[0]  
        possible_files = glob.glob(os.path.join(self.img_dir, f"{base_name}.*"))  
        
        for file in possible_files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')): 
                return file
        return None  

    def _filter_valid_images(self):
        valid_indices = []
        for idx in range(len(self.data)):
            img_name = self.data.iloc[idx, 0]
            img_path = self._find_correct_image_path(img_name)
            
            if img_path:
                try:
                    with Image.open(img_path) as img:
                        img.verify()  
                    valid_indices.append(idx)
                except (UnidentifiedImageError, OSError):
                    print(f"Skipping corrupt image: {img_path}")
        return valid_indices

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        valid_idx = self.valid_indices[idx]
        img_name = self.data.iloc[valid_idx, 0]
        img_path = self._find_correct_image_path(img_name)
        if not img_path:
            random_idx = random.choice(self.valid_indices)
            img_name = self.data.iloc[random_idx, 0]
            img_path = self._find_correct_image_path(img_name)
        label = self.data.iloc[valid_idx, 3]  
        try:
            image = Image.open(img_path).convert("RGB")
        except (UnidentifiedImageError, OSError):
            print(f"Skipping unreadable image: {img_path}")
            random_idx = random.choice(self.valid_indices)
            random_img_name = self.data.iloc[random_idx, 0]
            random_img_path = self._find_correct_image_path(random_img_name)
            image = Image.open(random_img_path).convert("RGB")
            label = self.data.iloc[random_idx, 3]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [17]:
csv_file = "/kaggle/input/multibully/Cyberbully_corrected_emotion_sentiment.xlsx - cyberbully.csv"  # Change to your CSV file path
img_directory = "/kaggle/input/multibully/bully_data/bully_data"  
dataset = CustomDataset(csv_file, img_directory, transform=transform)
num_classes = 2  
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
config = ViTConfig.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=num_classes)
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", config=config)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
scheduler = StepLR(optimizer, step_size=1, gamma=0.9)

Skipping corrupt image: /kaggle/input/multibully/bully_data/bully_data/411.jpg
Skipping corrupt image: /kaggle/input/multibully/bully_data/bully_data/2655.jpg
Skipping corrupt image: /kaggle/input/multibully/bully_data/bully_data/2663.jpg
Skipping corrupt image: /kaggle/input/multibully/bully_data/bully_data/2665.jpg
Skipping corrupt image: /kaggle/input/multibully/bully_data/bully_data/2705.jpg
Skipping corrupt image: /kaggle/input/multibully/bully_data/bully_data/2707.jpg
Skipping corrupt image: /kaggle/input/multibully/bully_data/bully_data/2723.jpg
Skipping corrupt image: /kaggle/input/multibully/bully_data/bully_data/2795.jpg
Skipping corrupt image: /kaggle/input/multibully/bully_data/bully_data/2874.jpg
Skipping corrupt image: /kaggle/input/multibully/bully_data/bully_data/3010.jpg
Skipping corrupt image: /kaggle/input/multibully/bully_data/bully_data/4150.jpg
Skipping corrupt image: /kaggle/input/multibully/bully_data/bully_data/5179.jpg
Skipping corrupt image: /kaggle/input/mul

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [12]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in tqdm(loader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU/CPU

        optimizer.zero_grad()
        outputs = model(inputs).logits  # Extract logits
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += len(labels)

    avg_loss = total_loss / len(loader)
    accuracy = correct_predictions / total_samples
    return avg_loss, accuracy


In [13]:
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).logits
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += len(labels)

    avg_loss = total_loss / len(loader)
    accuracy = correct_predictions / total_samples
    return avg_loss, accuracy


In [15]:
num_epochs = 20
early_stop_patience = 3
best_valid_loss = float("inf")
epochs_since_improvement = 0
inputs, labels = inputs.to(device), labels.to(device)

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, device)
    valid_loss, valid_accuracy = evaluate(model, valid_loader, criterion, device)

    print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.4f}")
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        epochs_since_improvement = 0
    else:
        epochs_since_improvement += 1

    if epochs_since_improvement == early_stop_patience:
        print("Early stopping triggered")
        break
    scheduler.step()


NameError: name 'inputs' is not defined

In [ ]:
model.save('/kaggle/working/vit_model')
torch.save(model.state_dict(), '/kaggle/working/vit_model_dict.pth')
model_save_path = "/kaggle/working/vit_model.pth"
torch.save(model, model_save_path)  
print(f"Model saved to {model_save_path}")


Final Model

In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.multiprocessing as mp
import pandas as pd
import numpy as np
import os
import pickle
import random
from PIL import Image, ImageOps, UnidentifiedImageError
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, ViTImageProcessor
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
csv_path = '/kaggle/input/multibully/Cyberbully_corrected_emotion_sentiment.xlsx - cyberbully.csv'
image_dir = '/kaggle/input/multibully/bullydata/bullydata'  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
roberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
text_model = XLMRobertaForSequenceClassification.from_pretrained("/kaggle/input/checkpoint/roberta_model1/kaggle/working/roberta_model1",from_tf=True)
text_model = text_model.to(device)
text_model.eval()

vit_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
vit_model = torch.load('/kaggle/input/checkpoint/vit_model.pth', map_location=device)
vit_model = vit_model.to(device)
vit_model.eval()

class TextAugmentation:
    def __init__(self, p=0.15):  
        self.p = p
    def augment(self, text):
        if random.random() > self.p:
            return text
        words = text.split()
        if len(words) <= 3:
            return text
        if len(words) > 3:
            del_idx = random.randint(0, len(words) - 1)
            words.pop(del_idx)
            
        return ' '.join(words)

class ImageAugmentation:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.2),  
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
        
        self.basic_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
    
    def augment(self, image, is_training=True):
        if is_training:
            return self.transform(image)
        return self.basic_transform(image)

def extract_text_features(text):
    inputs = roberta_tokenizer(
        text, add_special_tokens=True, max_length=512,
        padding='max_length', truncation=True, return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        outputs = text_model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]
        feature_vector = hidden_states[:, 0, :]
    
    noise = np.random.normal(0, 0.05, feature_vector.shape)
    feature_vector = feature_vector.cpu().numpy() + noise
    return feature_vector.squeeze(0)

def extract_image_features(image_path, augment=False):
    img_aug = ImageAugmentation()
    try:
        image = Image.open(image_path).convert('RGB')
        if augment:
            img_tensor = img_aug.augment(image, is_training=True).unsqueeze(0).to(device)
        else:
            img_tensor = img_aug.augment(image, is_training=False).unsqueeze(0).to(device)
            
        with torch.no_grad():
            outputs = vit_model(img_tensor)
        feature_vector = outputs.squeeze(0).cpu().numpy()
        noise = np.random.normal(0, 0.05, feature_vector.shape)
        return feature_vector + noise
    except (FileNotFoundError, UnidentifiedImageError, OSError) as e:
        print(f"Skipping image {image_path}: {e}")
        return np.zeros(768)

def extract_metadata_features(row):
    features = []
    sentiment_map = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
    sentiment = sentiment_map.get(row.get('Sentiment', 'Neutral'), 0)
    features.append(sentiment)
    emotions = ['Disgust', 'Surprise', 'Happiness', 'Angry', 'Fear'] 
    emotion = row.get('Emotion', 'Neutral')
    emotion_vec = [1 if emotion == e else 0 for e in emotions]
    features.extend(emotion_vec)
    sarcasm = 1 if row.get('Sarcasm', 'No') == 'Yes' else 0
    features.append(sarcasm)
    noise = np.random.normal(0, 0.1, len(features))
    features = np.array(features, dtype=np.float32) + noise
    return features

def save_text_features(csv_path, save_path):
    df = pd.read_csv(csv_path).dropna(subset=['Img-Text-Label'])
    text_feats = {}
    for i, row in df.iterrows():
        text = row['Img-Text']
        text_feats[text] = extract_text_features(text)
        if i % 100 == 0:
            print(f"Processed {i}/{len(df)} text samples")
    with open(save_path, 'wb') as f:
        pickle.dump(text_feats, f)
    print(f"Text features saved to {save_path}")

def save_image_features(csv_path, image_dir, save_path):
    df = pd.read_csv(csv_path).dropna(subset=['Img-Text-Label'])
    image_feats = {}
    for i, row in enumerate(df['Img-Name'].tolist()):
        img_path = os.path.join(image_dir, row)
        image_feats[row] = extract_image_features(img_path) if os.path.exists(img_path) else np.zeros(768)
        if i % 100 == 0:
            print(f"Processed {i}/{len(df)} image samples")
    np.save(save_path, image_feats)
    print(f"Image features saved to {save_path}")

def save_metadata_features(csv_path, save_path):
    df = pd.read_csv(csv_path).dropna(subset=['Img-Text-Label'])
    metadata_feats = {}
    for i, row in df.iterrows():
        img_name = row['Img-Name']
        metadata_feats[img_name] = extract_metadata_features(row)
        if i % 100 == 0:
            print(f"Processed {i}/{len(df)} metadata samples")
    with open(save_path, 'wb') as f:
        pickle.dump(metadata_feats, f)
    print(f"Metadata features saved to {save_path}")

class CyberbullyingDataset(Dataset):
    def __init__(self, dataframe, text_feat_path, img_feat_path, metadata_feat_path, label_map, 
                 augment=False, text_augmentor=None):
        self.data = dataframe
        self.label_map = label_map
        self.augment = augment
        self.text_augmentor = text_augmentor if text_augmentor and augment else None
        with open(text_feat_path, 'rb') as f:
            self.text_feats = pickle.load(f)
        self.image_feats = np.load(img_feat_path, allow_pickle=True).item()
        
        with open(metadata_feat_path, 'rb') as f:
            self.metadata_feats = pickle.load(f)

        self.texts = self.data['Img-Text'].tolist()
        self.labels = [label_map[label] for label in self.data['Img-Text-Label'].tolist()]
        self.image_names = self.data['Img-Name'].tolist()
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        img_name = self.image_names[idx]
        label = self.labels[idx]
        text_feat = self.text_feats.get(text, np.zeros(768))
        img_feat = self.image_feats.get(img_name, np.zeros(768))
        meta_feat = self.metadata_feats.get(img_name, np.zeros(7))  
        return (
            torch.tensor(text_feat, dtype=torch.float32),
            torch.tensor(img_feat, dtype=torch.float32),
            torch.tensor(meta_feat, dtype=torch.float32),
            torch.tensor(label, dtype=torch.long)
        )

class GatedFusion(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.text_gate = nn.Linear(dim, dim)
        self.image_gate = nn.Linear(dim, dim)
        self.fusion_gate = nn.Linear(dim * 2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, text_feats, image_feats):
        text_gate = self.sigmoid(self.text_gate(text_feats))
        image_gate = self.sigmoid(self.image_gate(image_feats))
        gated_text = text_feats * text_gate
        gated_image = image_feats * image_gate
        combined = torch.cat([text_feats, image_feats], dim=1)
        fusion_weight = self.sigmoid(self.fusion_gate(combined))
        fusion = fusion_weight * gated_text + (1 - fusion_weight) * gated_image
        return fusion

class CyberbullyingClassifier(nn.Module):
    def __init__(self, text_dim=768, img_dim=768, meta_dim=7):  
        super().__init__()
        self.feature_dim = text_dim
        self.fusion = GatedFusion(text_dim)
        self.meta_encoder = nn.Sequential(
            nn.Linear(meta_dim, 32),  
            nn.ReLU()
        )
        self.fc1 = nn.Linear(text_dim, 256)  
        self.fc2 = nn.Linear(256 + 32, 128)  
        self.classifier = nn.Linear(128, 2)
        self.dropout1 = nn.Dropout(0.1)  
        self.dropout2 = nn.Dropout(0.1)  
        self.relu = nn.ReLU()

    def forward(self, text_feats, image_feats, meta_feats):
        meta_encoded = self.meta_encoder(meta_feats)
        fused_features = self.fusion(text_feats, image_feats)
        x = self.relu(self.fc1(fused_features))
        x = self.dropout1(x)
        x = torch.cat([x, meta_encoded], dim=1)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        logits = self.classifier(x)
        return logits

def train_and_evaluate(csv_path, text_feat_path, img_feat_path, metadata_feat_path, 
                       batch_size=32, epochs=100, lr=0.001, test_size=0.2):  # Fewer epochs, higher lr
    if torch.cuda.is_available():
        torch.multiprocessing.set_start_method('spawn', force=True)
    df = pd.read_csv(csv_path).dropna(subset=['Img-Text-Label'])
    df = df[df['Img-Text-Label'].isin(['Bully', 'Nonbully'])]
    
    label_map = {"Bully": 1, "Nonbully": 0}
    text_augmentor = TextAugmentation(p=0.15)  
    train_df, test_df = train_test_split(
        df, test_size=test_size, random_state=42, 
        stratify=df['Img-Text-Label']
    )
    print(f"Training samples: {len(train_df)}, Test samples: {len(test_df)}")
    print(f"Training class distribution: {train_df['Img-Text-Label'].value_counts()}")
    print(f"Test class distribution: {test_df['Img-Text-Label'].value_counts()}")
    train_dataset = CyberbullyingDataset(
        train_df, text_feat_path, img_feat_path, metadata_feat_path, 
        label_map, augment=False, text_augmentor=None
    )
    
    test_dataset = CyberbullyingDataset(
        test_df, text_feat_path, img_feat_path, metadata_feat_path, 
        label_map, augment=False
    )
    
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, 
        num_workers=0
    )
    
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, 
        num_workers=0
    )
    
    with open(metadata_feat_path, 'rb') as f:
        metadata_feats = pickle.load(f)
        sample_key = list(metadata_feats.keys())[0]
        meta_dim = metadata_feats[sample_key].shape[0]
        print(f"Using metadata dimension: {meta_dim}")
    
    model = CyberbullyingClassifier(meta_dim=meta_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.7, patience=2, verbose=True
    )
    best_accuracy = 0
    patience = 10
    trigger_times = 0
    for epoch in range(epochs):
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        for text_feats, image_feats, meta_feats, labels in train_loader:
            text_feats = text_feats.to(device)
            image_feats = image_feats.to(device)
            meta_feats = meta_feats.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(text_feats, image_feats, meta_feats)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item() * labels.size(0)
            train_correct += (outputs.argmax(1) == labels).sum().item()
            train_total += labels.size(0)
        
        train_loss = train_loss / train_total
        train_accuracy = train_correct / train_total
        model.eval()
        test_loss, test_correct, test_total = 0, 0, 0
        test_preds, test_true = [], []
        with torch.no_grad():
            for text_feats, image_feats, meta_feats, labels in test_loader:
                text_feats = text_feats.to(device)
                image_feats = image_feats.to(device)
                meta_feats = meta_feats.to(device)
                labels = labels.to(device)
                
                outputs = model(text_feats, image_feats, meta_feats)
                loss = criterion(outputs, labels)
                
                test_loss += loss.item() * labels.size(0)
                test_preds.extend(outputs.argmax(1).cpu().numpy())
                test_true.extend(labels.cpu().numpy())
                test_correct += (outputs.argmax(1) == labels).sum().item()
                test_total += labels.size(0)
        
        test_loss = test_loss / test_total
        test_accuracy = test_correct / test_total
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}")
        print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}")
        scheduler.step(test_accuracy)
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            trigger_times = 0
            torch.save(model.state_dict(), "cyberbullying_model_best.pth")
            print(f"Model saved with test accuracy: {test_accuracy:.4f}")
            
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    model.load_state_dict(torch.load("cyberbullying_model_best.pth"))
    model.eval()
    test_correct, test_total = 0, 0
    test_preds, test_true = [], []
    
    with torch.no_grad():
        for text_feats, image_feats, meta_feats, labels in test_loader:
            text_feats = text_feats.to(device)
            image_feats = image_feats.to(device)
            meta_feats = meta_feats.to(device)
            labels = labels.to(device)
            
            outputs = model(text_feats, image_feats, meta_feats)
            
            test_preds.extend(outputs.argmax(1).cpu().numpy())
            test_true.extend(labels.cpu().numpy())
            test_correct += (outputs.argmax(1) == labels).sum().item()
            test_total += labels.size(0)
    
    final_accuracy = test_correct / test_total
    print("\nFinal Test Results:")
    print(f"Accuracy: {final_accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(test_true, test_preds, target_names=['Nonbully', 'Bully']))
    print("\nConfusion Matrix:")
    cm = confusion_matrix(test_true, test_preds)
    print(cm)
    return model
def train_and_evaluate_kfold(csv_path, text_feat_path, img_feat_path, metadata_feat_path, 
                         batch_size=32, epochs=100, lr=0.001, n_splits=5):
    if torch.cuda.is_available():
        torch.multiprocessing.set_start_method('spawn', force=True)
    df = pd.read_csv(csv_path).dropna(subset=['Img-Text-Label'])
    df = df[df['Img-Text-Label'].isin(['Bully', 'Nonbully'])]
    label_map = {"Bully": 1, "Nonbully": 0}
    text_augmentor = TextAugmentation(p=0.15)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_accuracies = []
    fold_reports = []
    fold_cms = []
    full_dataset = df.reset_index(drop=True)
    with open(metadata_feat_path, 'rb') as f:
        metadata_feats = pickle.load(f)
        sample_key = list(metadata_feats.keys())[0]
        meta_dim = metadata_feats[sample_key].shape[0]
        print(f"Detected metadata feature dimension: {meta_dim}")
    for fold, (train_idx, test_idx) in enumerate(kf.split(full_dataset)):
        print(f"\n{'='*50}")
        print(f"FOLD {fold+1}/{n_splits}")
        print(f"{'='*50}")
        train_df = full_dataset.iloc[train_idx]
        test_df = full_dataset.iloc[test_idx]
        print(f"Training samples: {len(train_df)}, Test samples: {len(test_df)}")
        print(f"Training class distribution: {train_df['Img-Text-Label'].value_counts()}")
        print(f"Test class distribution: {test_df['Img-Text-Label'].value_counts()}")
        train_dataset = CyberbullyingDataset(
            train_df, text_feat_path, img_feat_path, metadata_feat_path, 
            label_map, augment=False, text_augmentor=None
        )
        
        test_dataset = CyberbullyingDataset(
            test_df, text_feat_path, img_feat_path, metadata_feat_path, 
            label_map, augment=False
        )
        train_loader = DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True, 
            num_workers=0
        )
        
        test_loader = DataLoader(
            test_dataset, batch_size=batch_size, shuffle=False, 
            num_workers=0
        )
        model = CyberbullyingClassifier(meta_dim=meta_dim).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.7, patience=2, verbose=True
        )
        best_accuracy = 0
        patience = 10
        trigger_times = 0
        for epoch in range(epochs):
            model.train()
            train_loss, train_correct, train_total = 0, 0, 0
            
            for text_feats, image_feats, meta_feats, labels in train_loader:
                text_feats = text_feats.to(device)
                image_feats = image_feats.to(device)
                meta_feats = meta_feats.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(text_feats, image_feats, meta_feats)
                loss = criterion(outputs, labels)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                train_loss += loss.item() * labels.size(0)
                train_correct += (outputs.argmax(1) == labels).sum().item()
                train_total += labels.size(0)
            
            train_loss = train_loss / train_total
            train_accuracy = train_correct / train_total
            model.eval()
            test_loss, test_correct, test_total = 0, 0, 0
            test_preds, test_true = [], []
            
            with torch.no_grad():
                for text_feats, image_feats, meta_feats, labels in test_loader:
                    text_feats = text_feats.to(device)
                    image_feats = image_feats.to(device)
                    meta_feats = meta_feats.to(device)
                    labels = labels.to(device)
                    
                    outputs = model(text_feats, image_feats, meta_feats)
                    loss = criterion(outputs, labels)
                    
                    test_loss += loss.item() * labels.size(0)
                    test_preds.extend(outputs.argmax(1).cpu().numpy())
                    test_true.extend(labels.cpu().numpy())
                    test_correct += (outputs.argmax(1) == labels).sum().item()
                    test_total += labels.size(0)
            
            test_loss = test_loss / test_total
            test_accuracy = test_correct / test_total
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}")
            print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}")
            scheduler.step(test_accuracy)
            if test_accuracy > best_accuracy:
                best_accuracy = test_accuracy
                trigger_times = 0
                torch.save(model.state_dict(), f"cyberbullying_model_fold{fold+1}_best.pth")
                print(f"Model saved with test accuracy: {test_accuracy:.4f}")
                
            else:
                trigger_times += 1
                if trigger_times >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
        model.load_state_dict(torch.load(f"cyberbullying_model_fold{fold+1}_best.pth"))
        model.eval()
        test_correct, test_total = 0, 0
        test_preds, test_true = [], []
        
        with torch.no_grad():
            for text_feats, image_feats, meta_feats, labels in test_loader:
                text_feats = text_feats.to(device)
                image_feats = image_feats.to(device)
                meta_feats = meta_feats.to(device)
                labels = labels.to(device)
                
                outputs = model(text_feats, image_feats, meta_feats)
                
                test_preds.extend(outputs.argmax(1).cpu().numpy())
                test_true.extend(labels.cpu().numpy())
                test_correct += (outputs.argmax(1) == labels).sum().item()
                test_total += labels.size(0)
        
        fold_accuracy = test_correct / test_total
        fold_accuracies.append(fold_accuracy)
        fold_report = classification_report(test_true, test_preds, target_names=['Nonbully', 'Bully'], output_dict=True)
        fold_reports.append(fold_report)
        fold_cm = confusion_matrix(test_true, test_preds)
        fold_cms.append(fold_cm)
        
        print(f"\nFold {fold+1} Results:")
        print(f"Accuracy: {fold_accuracy:.4f}")
        print("\nClassification Report:")
        print(classification_report(test_true, test_preds, target_names=['Nonbully', 'Bully']))
        
        print("\nConfusion Matrix:")
        print(fold_cm)
    avg_accuracy = sum(fold_accuracies) / len(fold_accuracies)
    print(f"\n{'='*50}")
    print(f"AVERAGE RESULTS ACROSS {n_splits} FOLDS")
    print(f"{'='*50}")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    avg_report = {}
    for label in ['Nonbully', 'Bully']:
        avg_report[label] = {
            'precision': sum(report[label]['precision'] for report in fold_reports) / n_splits,
            'recall': sum(report[label]['recall'] for report in fold_reports) / n_splits,
            'f1-score': sum(report[label]['f1-score'] for report in fold_reports) / n_splits
        }
    
    avg_report['macro avg'] = {
        'precision': sum(report['macro avg']['precision'] for report in fold_reports) / n_splits,
        'recall': sum(report['macro avg']['recall'] for report in fold_reports) / n_splits,
        'f1-score': sum(report['macro avg']['f1-score'] for report in fold_reports) / n_splits
    }
    
    avg_report['weighted avg'] = {
        'precision': sum(report['weighted avg']['precision'] for report in fold_reports) / n_splits,
        'recall': sum(report['weighted avg']['recall'] for report in fold_reports) / n_splits,
        'f1-score': sum(report['weighted avg']['f1-score'] for report in fold_reports) / n_splits
    }
    
    print("\nAverage Classification Report:")
    print(f"               precision    recall  f1-score")
    print(f"Nonbully       {avg_report['Nonbully']['precision']:.4f}      {avg_report['Nonbully']['recall']:.4f}    {avg_report['Nonbully']['f1-score']:.4f}")
    print(f"Bully          {avg_report['Bully']['precision']:.4f}      {avg_report['Bully']['recall']:.4f}    {avg_report['Bully']['f1-score']:.4f}")
    print(f"macro avg      {avg_report['macro avg']['precision']:.4f}      {avg_report['macro avg']['recall']:.4f}    {avg_report['macro avg']['f1-score']:.4f}")
    print(f"weighted avg   {avg_report['weighted avg']['precision']:.4f}      {avg_report['weighted avg']['recall']:.4f}    {avg_report['weighted avg']['f1-score']:.4f}")

    avg_cm = np.mean(fold_cms, axis=0).astype(int)
    print("\nAverage Confusion Matrix:")
    print(avg_cm)
    
    return fold_accuracies, fold_reports, fold_cms
if __name__ == "__main__":
    text_feat_path = "text_features.pkl"
    img_feat_path = "image_features.npy"
    metadata_feat_path = "metadata_features.pkl"
    if not os.path.exists(text_feat_path):
        print("Extracting text features...")
        save_text_features(csv_path, text_feat_path)
    
    if not os.path.exists(img_feat_path):
        print("Extracting image features...")
        save_image_features(csv_path, image_dir, img_feat_path)
    
    if not os.path.exists(metadata_feat_path):
        print("Extracting metadata features...")
        save_metadata_features(csv_path, metadata_feat_path)
    print("Starting K-fold cross-validation training...")
    fold_accuracies, fold_reports, fold_cms = train_and_evaluate_kfold(
        csv_path=csv_path,
        text_feat_path=text_feat_path,
        img_feat_path=img_feat_path,
        metadata_feat_path=metadata_feat_path,
        batch_size=32,
        epochs=100,
        lr=0.001,
        n_splits=5
    )
    print("\nTraining final model on full dataset...")
    final_model = train_and_evaluate(
        csv_path=csv_path,
        text_feat_path=text_feat_path,
        img_feat_path=img_feat_path,
        metadata_feat_path=metadata_feat_path,
        batch_size=32,
        epochs=100,
        lr=0.001
    )
    torch.save(final_model.state_dict(), "cyberbullying_model_final.pth")
    print("Final model saved to cyberbullying_model_final.pth")
    print("Training and evaluation complete!")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing XLMRobertaForSequenceClassification.

All the weights of XLMRobertaForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use XLMRobertaForSequenceClassification for predictions without further training.


...

<ipython-input-1-f083a802e9dc>:681: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"cyberbullying_model_fold{fold+1}_best.pth"))


Epoch 1/100
Train Loss: 0.4761, Train Acc: 0.7829
Test Loss: 0.4316, Test Acc: 0.8162
Model saved with test accuracy: 0.8162
Epoch 2/100
Train Loss: 0.4056, Train Acc: 0.8215
Test Loss: 0.4222, Test Acc: 0.8231
Model saved with test accuracy: 0.8231
Epoch 3/100
Train Loss: 0.3979, Train Acc: 0.8222
Test Loss: 0.4365, Test Acc: 0.8188
Epoch 4/100
Train Loss: 0.3955, Train Acc: 0.8258
Test Loss: 0.4203, Test Acc: 0.8205
Epoch 5/100
Train Loss: 0.3900, Train Acc: 0.8288
Test Loss: 0.4324, Test Acc: 0.8162
Epoch 6/100
Train Loss: 0.3835, Train Acc: 0.8316
Test Loss: 0.4179, Test Acc: 0.8197
Epoch 7/100
Train Loss: 0.3802, Train Acc: 0.8318
Test Loss: 0.4282, Test Acc: 0.8205
Epoch 8/100
Train Loss: 0.3791, Train Acc: 0.8324
Test Loss: 0.4185, Test Acc: 0.8214
Epoch 9/100
Train Loss: 0.3725, Train Acc: 0.8397
Test Loss: 0.4308, Test Acc: 0.8205
Epoch 10/100
Train Loss: 0.3648, Train Acc: 0.8429
Test Loss: 0.4406, Test Acc: 0.8145
Epoch 11/100
Train Loss: 0.3594, Train Acc: 0.8442
Test Loss:

<ipython-input-1-f083a802e9dc>:681: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"cyberbullying_model_fold{fold+1}_best.pth"))


Using metadata dimension: 7
Epoch 1/100
Train Loss: 0.4815, Train Acc: 0.7798
Test Loss: 0.4099, Test Acc: 0.8138
Model saved with test accuracy: 0.8138
Epoch 2/100
Train Loss: 0.4116, Train Acc: 0.8196
Test Loss: 0.4035, Test Acc: 0.8181
Model saved with test accuracy: 0.8181
Epoch 3/100
Train Loss: 0.4046, Train Acc: 0.8253
Test Loss: 0.4104, Test Acc: 0.8215
Model saved with test accuracy: 0.8215
Epoch 4/100
Train Loss: 0.4025, Train Acc: 0.8264
Test Loss: 0.4088, Test Acc: 0.8096
Epoch 5/100
Train Loss: 0.3971, Train Acc: 0.8285
Test Loss: 0.4503, Test Acc: 0.8053
Epoch 6/100
Train Loss: 0.3965, Train Acc: 0.8305
Test Loss: 0.4060, Test Acc: 0.8164
Epoch 7/100
Train Loss: 0.3910, Train Acc: 0.8305
Test Loss: 0.4072, Test Acc: 0.8207
Epoch 8/100
Train Loss: 0.3893, Train Acc: 0.8285
Test Loss: 0.4071, Test Acc: 0.8104
Epoch 9/100
Train Loss: 0.3866, Train Acc: 0.8294
Test Loss: 0.4054, Test Acc: 0.8147
Epoch 10/100
Train Loss: 0.3811, Train Acc: 0.8332
Test Loss: 0.4144, Test Acc: 0

<ipython-input-1-f083a802e9dc>:478: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("cyberbullying_model_best.pth"))
